In [15]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objects as go
import plotly.io as pio

# Define the correct column names
column_names = ['ET', 'V', 'N', 'SA', 'IA', 'RL', 'RE', 'P', 'FX', 'FY', 'FZ', 'MX', 'MZ',
                'NFX', 'NFY', 'RST', 'TSTI', 'TSTC', 'TSTO', 'AMBTMP', 'SR']

# Function to set the top row as header and reset index
def set_top_row_as_header(df):
    df.columns = df.iloc[0].values
    df = df.drop(df.index[0])
    df = df.reset_index(drop=True)
    return df

# Try reading the file with flexible error handling
def load_and_prepare_dataframe(file_path):
    try:
        # Read with flexible handling of bad lines (skip rows with mismatched columns)
        df = pd.read_csv(file_path, delim_whitespace=True, names=column_names, on_bad_lines='skip')
        print(f"Successfully loaded {file_path}")
    except Exception as e:
        print(f"Failed to load {file_path}. Error: {e}")
        return None

    # Drop the first two rows and reset index
    df = df.iloc[2:].reset_index(drop=True)

    # Drop dataframes with zero length
    if len(df) > 0:
        return df
    else:
        print(f"File {file_path} has zero length after preprocessing.")
        return None

# Function to process data and generate 3D interactive plots for each file path
def process_and_save_3d_plotly(file_paths, output_directory):
    for file_path in file_paths:
        # Load and prepare the dataframe
        df = load_and_prepare_dataframe(file_path)
        if df is None:
            continue

        print(f"Processing file: {file_path}")

        # Convert columns to numeric and clean data
        df['MZ'] = pd.to_numeric(df['MZ'], errors='coerce')
        df['SA'] = pd.to_numeric(df['SA'], errors='coerce')
        df['FZ'] = pd.to_numeric(df['FZ'], errors='coerce')
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(subset=['MZ', 'SA', 'FZ'], inplace=True)

        # Interpolation grid setup
        grid_x, grid_y = np.meshgrid(
            np.linspace(df['SA'].min(), df['SA'].max(), 50),
            np.linspace(df['FZ'].min(), df['FZ'].max(), 50)
        )
        grid_z = griddata((df['SA'], df['FZ']), df['MZ'], (grid_x, grid_y), method='cubic')

        # Create the interactive 3D surface plot using Plotly
        fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale='Viridis')])

        # Set plot titles and labels
        fig.update_layout(
            title=f'Slip Angle vs. Vertical Load vs. MZ for {os.path.basename(file_path)}',
            scene=dict(
                xaxis_title='Slip Angle (deg)',
                yaxis_title='Vertical Load (N)',
                zaxis_title='Alignment Torque (Nm)'
            ),
            autosize=True,
            width=800,
            height=800,
            margin=dict(l=65, r=50, b=65, t=90)
        )

        # Save the interactive plot as an HTML file
        html_file_path = f"{output_directory}/{os.path.basename(file_path).replace('.dat', '')}_mz.html"
        pio.write_html(fig, file=html_file_path, auto_open=False)

        print(f"Saved 3D plot to: {html_file_path}")


In [16]:
# Example usage on a list of file paths
file_paths = [
    '/Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw11.dat',
    '/Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw15.dat',
    '/Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat',
]

# Specify the output directory for saving the HTML files
output_directory = "/Users/peytonboone/Desktop/feb/fall2024/MzvsSAvsFZ"

# Call the function to process and save 3D graphs as HTML files using Plotly
process_and_save_3d_plotly(file_paths, output_directory)

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Successfully loaded /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw11.dat
Processing file: /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw11.dat
Saved 3D plot to: /Users/peytonboone/Desktop/feb/fall2024/MzvsSAvsFZ/B2356raw11_mz.html


/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Successfully loaded /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw15.dat
Processing file: /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw15.dat
Saved 3D plot to: /Users/peytonboone/Desktop/feb/fall2024/MzvsSAvsFZ/B2356raw15_mz.html
Successfully loaded /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat


/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_11930/2243151223.py:23: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Processing file: /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat
Saved 3D plot to: /Users/peytonboone/Desktop/feb/fall2024/MzvsSAvsFZ/B2356raw4_mz.html
